<a href="https://colab.research.google.com/github/Aditya-Patel/Stat598-FinalProject/blob/main/MPLA_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dataset
import yfinance as yf

# Visualization
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import seaborn as sns

# Date Manipulation
from datetime import datetime

# PyTorch Libraries
import tensorflow as tf
from tensorflow import keras

# statstical testing, plotting and decompositions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

from scipy import stats

# Set device usage to GPU if available
device = tf.device('/device:gpu:1')

In [2]:
start_date = '2020-06-01'
end_date = '2023-12-01'
etf_ticker = 'MLPA'
moving_average_list = []
etf_tickers_url = "https://raw.githubusercontent.com/Aditya-Patel/Stat598-FinalProject/main/mlpa_full-holdings.csv"
crude_oil_stock_url = "https://raw.githubusercontent.com/Aditya-Patel/Stat598-FinalProject/main/crude%20oil%20spot%20price.csv"

<h1>Create joint dataset between spot price and ETF Data</h1>

In [3]:
df_holdings = pd.read_csv(etf_tickers_url)
df_holdings = df_holdings[(df_holdings['Name'] != 'OTHER PAYABLE & RECEIVABLES') & (df_holdings['Name'] != 'CASH')]
df_holdings[f'Market Value ($)'] = df_holdings[f'Market Value ($)'].str.replace(',', '').astype(float)
total_market_value = df_holdings[f'Market Value ($)'].sum()
df_holdings['Percentage Holdings By Value'] = (df_holdings[f'Market Value ($)'] / total_market_value)

df_crude_price = pd.read_csv(crude_oil_stock_url,usecols=[0, 1])
df_crude_price['Date'] = pd.to_datetime(df_crude_price['Date'], format='%b %d, %Y')
df_crude_price.set_index('Date', inplace=True)
df_crude_price.rename(columns={'WTI Barrell Spot Price':'Spot Price'}, inplace=True)
df_crude_price['Spot Price'] = df_crude_price['Spot Price'].fillna(method='ffill')

all_stocks_data = yf.download(etf_ticker, start=start_date, end=end_date)

# Join spot price and etf data
all_stocks_data['Ticker'] = etf_ticker
all_stocks_data.columns = [f'{etf_ticker}_{col}' if col not in ['Ticker', 'Date'] else col for col in all_stocks_data.columns]
all_stocks_data = all_stocks_data.join(df_crude_price, how='left')

[*********************100%%**********************]  1 of 1 completed


<h1>Load all tickers within the ETF</h1>

In [4]:
# load all tickers part of that etf
for ticker in df_holdings['Ticker']:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    percentage_holding = df_holdings.loc[df_holdings['Ticker'] == ticker, 'Percentage Holdings By Value'].iloc[0]
    all_stocks_data[f'{ticker}_Percent_Holding'] = percentage_holding
    stock_data.columns = [f'{ticker}_{col}' if col != 'Ticker' else col for col in stock_data.columns]
    all_stocks_data = all_stocks_data.join(stock_data, how='outer')

all_stocks_data.fillna(0, inplace=True)
sum_values = pd.Series(0, index=all_stocks_data.index)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
# Get all closing values

all_stocks_data[f'{etf_ticker}_Next_Close'] = all_stocks_data[f'{etf_ticker}_Close'].shift(-1)
all_stocks_data = all_stocks_data.drop(all_stocks_data.index[-1])
close_values = all_stocks_data[[col for col in all_stocks_data.columns if '_Close' in col or col == 'Spot Price' or col == f'{etf_ticker}_Next_Close']]


,MLPA_Close,Spot Price,ET_Close,EPD_Close,MPLX_Close,PAA_Close,WES_Close,ENLC_Close,CQP_Close,NS_Close,...,PSX_Close,MLPX_Close,CAPL_Close,SMLP_Close,GPP_Close,NGL_Close,MMLP_Close,SNMP_Close,USDP_Close,MLPA_Next_Close
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-01,29.459999,35.49,8.21,19.270000,18.580000,9.87,8.830000,2.46,34.910000,16.870001,...,78.830002,25.879999,15.770000,22.049999,6.450,5.73,2.46,0.000,3.370,30.150000
2020-06-02,30.150000,36.88,8.31,19.680000,19.000000,10.17,9.110000,2.57,36.250000,17.020000,...,81.570000,26.430000,17.000000,24.600000,6.650,5.84,2.62,0.000,3.170,31.190001
2020-06-03,31.190001,37.33,8.53,20.100000,19.730000,10.50,9.400000,2.71,37.230000,17.400000,...,83.730003,27.170000,16.850000,26.100000,7.185,6.40,2.62,0.000,3.260,31.940001
2020-06-04,31.940001,37.42,8.82,20.400000,20.150000,10.86,9.700000,2.76,37.700001,17.670000,...,83.129997,27.459999,16.719999,24.000000,7.350,6.36,2.68,0.000,3.420,33.560001
2020-06-05,33.560001,39.49,9.16,20.950001,20.950001,11.36,11.200000,3.41,39.279999,18.370001,...,87.400002,28.840000,15.780000,25.200001,8.800,6.93,2.80,0.000,3.610,34.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21,45.020000,78.35,13.70,26.549999,35.720001,15.56,28.500000,12.94,57.680000,18.070000,...,117.410004,43.930000,22.260000,18.250000,12.520,4.11,2.36,1.490,0.123,45.200001
2023-11-22,45.200001,76.80,13.74,26.700001,35.779999,15.57,28.440001,13.15,58.299999,18.340000,...,118.010002,44.090000,22.440001,18.799999,12.440,4.09,2.42,1.560,0.250,45.200001
2023-11-24,45.200001,74.83,13.72,26.740000,35.930000,15.60,28.450001,13.27,58.880001,18.660000,...,118.629997,44.299999,22.520000,19.129999,12.310,4.06,2.42,1.520,0.225,45.250000
